In [7]:
import json
import sqlite3
import re

# перенести в константы
FILEPATH = 'C:/Users/User/Desktop/Python/PythonProjects/Twitter_info_project/three_minutes_tweets.json.txt'
FILEPATH_AF = 'C:/Users/User/Desktop/Python/PythonProjects/Twitter_info_project/AFINN-111.txt'
db = 'mts_twits.db'

class Sqlite_Contextman:
    
    def __init__(self, dbname):
        self.dbname = dbname
        
    def __enter__(self):
        self.conn = sqlite3.connect(self.dbname)
        return self.conn
    
    def __exit__(self, exc_val, exc_type, exc_tb):
        self.conn.close()
        if exc_val:
            raise
    
    @staticmethod
    def db_querrier(cursor, sql):
        """Для запуска произвольного запроса"""
        cursor.execute(sql)

In [8]:
# запросы на создание таблиц и индексов
sql = ("""CREATE TABLE IF NOT EXISTS users (
user_id int NOT NULL PRIMARY KEY, 
username text, 
url text, 
location text,
lang text
);""",
       """CREATE TABLE IF NOT EXISTS twitts (
tweet_id int NOT NULL PRIMARY KEY, 
created_at text,
user_id int,
tweet_text text, 
country_code text, 
sentiment int
);""")

In [9]:
def sentiment_dict_maker(FILEPATH):
    "Возвращает словарь для эмоциональной окраски"
    word_dict = dict()
    """Читает маркеры эмоциональной окраски из файла и записывает их в БД"""
    line = True
    with open(FILEPATH_AF) as filetoread:
        while line != '':
            line = filetoread.readline()
            if line != '':
                sentiment = re.split('\t', line)
                sentiment[1] = int(sentiment[1])
                word_dict.update({sentiment[0] : sentiment[1]})
    return word_dict

In [10]:
def sentiment_counter(s, word_dict):
    """Считает количество вхождений подстроки в словарь"""
    res = 0
    s = s.split()
    for x in s:
        if x in word_dict:
            res += word_dict[x]
    return(res)

def tweets_inserter_sql(cursor, line_to_json, sentiment):
    """Функция для записи json в таблицу twitts"""
    country_code = None
    if line_to_json['place'] != None:
        country_code = line_to_json['place']['country_code']
    cursor.execute("""insert or ignore into twitts (tweet_id, created_at, user_id, tweet_text, country_code, sentiment) 
            values (?,?,?,?,?,?)""",[line_to_json['id'],
                                   line_to_json['created_at'],
                                   line_to_json['user']['id'],
                                   line_to_json['text'],
                                   country_code,
                                   sentiment])
    
def users_inserter_sql(cursor, line_to_json):
    """Функция для записи json в таблицу twitts"""
    cursor.execute("""insert or ignore into users (user_id, url, username, lang, location) 
            values (?,?,?,?,?)""",[line_to_json['user']['id'],
                                    line_to_json['user']['url'],
                                    line_to_json['user']['name'],
                                    line_to_json['user']['lang'],
                                    line_to_json['user']['location']])

def tweets_inserter(db, FILEPATH, cursor):
    """Читает строки из файла с твитами и вызывает метод записи в БД"""
    line = True
    with open(FILEPATH) as filetoread:
        while line != '':
            line = filetoread.readline()
            if line != '': 
                line_to_json = json.loads(line)
            if 'delete' not in line_to_json.keys():
                sentiment = sentiment_counter(line_to_json['text'], word_dict)
                tweets_inserter_sql(cursor, line_to_json, sentiment)
                users_inserter_sql(cursor, line_to_json)

In [11]:
# получаем словарь с эмоциональной окраской
word_dict = sentiment_dict_maker(FILEPATH_AF)  

In [13]:
with Sqlite_Contextman(db) as conn:
        cursor = conn.cursor()
        tweets_inserter(db, FILEPATH, cursor)
        conn.commit()